<a href="https://colab.research.google.com/github/Amisha1019/Traslation-intership-of-nullclass/blob/main/task_2_eng_hind.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets sentencepiece gradio --quiet


In [2]:
from datasets import Dataset

data = {
    "en": [
        "this is a beautiful day",
        "machine learning is powerful",
        "translation models are useful",
        "we are training our own model"
    ],
    "fr": [
        "cest une belle journee",
        "lapprentissage automatique est puissant",
        "les modeles de traduction sont utiles",
        "nous entrainons notre propre modele"
    ],
    "hi": [
        "यह एक सुंदर दिन है",
        "मशीन लर्निंग शक्तिशाली है",
        "अनुवाद मॉडल उपयोगी हैं",
        "हम अपना खुद का मॉडल प्रशिक्षित कर रहे हैं"
    ]
}

dataset = Dataset.from_dict(data)


In [3]:
from transformers import MarianTokenizer, MarianMTModel


In [4]:
# English → French
tokenizer_en_fr = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-fr")
model_en_fr = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-fr")

# English → Hindi
tokenizer_en_hi = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-hi")
model_en_hi = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-hi")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/306M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/306M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [5]:
def preprocess_en_fr(batch):
    inputs = tokenizer_en_fr(batch["en"], truncation=True, padding="max_length", max_length=64)
    labels = tokenizer_en_fr(batch["fr"], truncation=True, padding="max_length", max_length=64)
    inputs["labels"] = labels["input_ids"]
    return inputs

def preprocess_en_hi(batch):
    inputs = tokenizer_en_hi(batch["en"], truncation=True, padding="max_length", max_length=64)
    labels = tokenizer_en_hi(batch["hi"], truncation=True, padding="max_length", max_length=64)
    inputs["labels"] = labels["input_ids"]
    return inputs


In [6]:
from transformers import Trainer, TrainingArguments

train_fr = dataset.map(preprocess_en_fr, batched=True)
train_hi = dataset.map(preprocess_en_hi, batched=True)


Map:   0%|          | 0/4 [00:00<?, ? examples/s]

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

In [7]:
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=2,
    num_train_epochs=1,
    logging_steps=10,
    save_strategy="no",
    report_to="none"
)


In [9]:
def dual_translate(text):
    text = text.strip()

    if len(text) < 10:
        return "upload again", "upload again"

    fr_inputs = tokenizer_en_fr(text, return_tensors="pt", truncation=True)
    hi_inputs = tokenizer_en_hi(text, return_tensors="pt", truncation=True)

    fr_out = model_en_fr.generate(**fr_inputs)
    hi_out = model_en_hi.generate(**hi_inputs)

    french = tokenizer_en_fr.decode(fr_out[0], skip_special_tokens=True)
    hindi = tokenizer_en_hi.decode(hi_out[0], skip_special_tokens=True)

    return french, hindi


In [10]:
import gradio as gr

interface = gr.Interface(
    fn=dual_translate,
    inputs=gr.Textbox(label="Enter English text (10+ letters)", placeholder="machine learning is powerful"),
    outputs=[
        gr.Textbox(label="French Translation"),
        gr.Textbox(label="Hindi Translation")
    ],
    title="Dual Language Translator (English → French & Hindi)",
    description="Only translates English text with 10 or more letters. Uses trained ML models."
)


In [11]:
if __name__ == "__main__":
    interface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4c575dee9afc54de5c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
